# 7 函数装饰器和闭包
## 7.1装饰器基础知识

In [3]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
target()

running target()


In [4]:
target

<function __main__.target()>

## 7.2 Python何时执行装饰器

In [5]:
!python registration.py

runnning register(<function f1 at 0x0000023927599168>)
runnning register(<function f2 at 0x00000239276CCD38>)
running main()
registry -> [<function f1 at 0x0000023927599168>, <function f2 at 0x00000239276CCD38>]
running f1
running f2
running f3


In [6]:
import registration

runnning register(<function f1 at 0x000002093B6BAAF8>)
runnning register(<function f2 at 0x000002093B6BA948>)


In [8]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

In [9]:
registration.f1()

running f1


In [10]:
registration.f1()

running f1


In [11]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 7.3使用装饰器改进‘策略’模式

In [12]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

## 7.4变量作用域规则

In [13]:
def f1(a):
    print(a)
    print(b)

In [14]:
f1(a)

NameError: name 'a' is not defined

In [15]:
b = 6
f1(3)

3
6


In [16]:
def f2(a):
    print(a)
    print(b)
    b=9

In [17]:
b = 6
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [18]:
def f3(a):
    global b
    print(a)
    print(b)
    b=9

In [19]:
b = 5
f3(3)

3
5


## 7.5 闭包

In [21]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_val):
        self.series.append(new_val)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
avg(12)

12.0

In [22]:
avg(11)

11.5

In [23]:
def make_averager():
    
    series = []
    
    def averager(new_val):
        series.append(new_val)
        total = sum(series)
        return total/len(series)
    return averager

In [24]:
avg = make_averager()
avg(12)

12.0

In [25]:
avg(11)

11.5

## 7.6 nonlocal声明

In [26]:
def make_averager():
    
    count = 0
    total = 0
    
    def averager(new_val):
        nonlocal count, total
        count += 1
        total += new_val
        return total/count
    return averager

In [27]:
avg = make_averager()
avg(12)

12.0

In [28]:
avg(11)

11.5

## 7.7实现一个简单的装饰器

In [29]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        print('[%0.8fs]%s(%s)-> %r'%(elapsed, name, arg_str, result))
        return result
    return clocked


In [30]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def fac(n):
    return 1 if n < 1 else n*fac(n-1)
snooze(0.123)
fac(6)

[0.12246520s]snooze(0.123)-> None
[0.00000110s]fac(0)-> 1
[0.00028870s]fac(1)-> 1
[0.00059350s]fac(2)-> 2
[0.00086320s]fac(3)-> 6
[0.00106390s]fac(4)-> 24
[0.00125940s]fac(5)-> 120
[0.00150490s]fac(6)-> 720


720

In [31]:
fac.__name__

'clocked'

In [34]:
import time 
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(','.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s = %r'%(k, w) for k,w in sorted(kwargs.items)]
            arg_lst.append(''.join(pairs))
        
        arg_str = ', '.join(arg_lst)
        print('[%0.8f]%s(%s) ->%r'%(elapsed, name, arg_str, result))
        return result
    return clocked

In [35]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def fac(n):
    return 1 if n < 1 else n*fac(n-1)
snooze(0.123)
fac(6)

[0.12300801]snooze(0.123) ->None
[0.00000000]fac(0) ->1
[0.00000000]fac(1) ->1
[0.00000000]fac(2) ->2
[0.00000000]fac(3) ->6
[0.00000000]fac(4) ->24
[0.00000000]fac(5) ->120
[0.00000000]fac(6) ->720


720

## 7.8 标准库中的装饰器
### 7.8.1使用functools.lru_cache做备忘

In [36]:
@clock
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

fib(6)

[0.00000000]fib(1) ->1
[0.00000000]fib(0) ->0
[0.00102878]fib(2) ->1
[0.00000000]fib(1) ->1
[0.00102878]fib(3) ->2
[0.00000000]fib(1) ->1
[0.00000000]fib(0) ->0
[0.00000000]fib(2) ->1
[0.00102878]fib(4) ->3
[0.00000000]fib(1) ->1
[0.00000000]fib(0) ->0
[0.00000000]fib(2) ->1
[0.00000000]fib(1) ->1
[0.00000000]fib(3) ->2
[0.00102878]fib(5) ->5
[0.00000000]fib(1) ->1
[0.00000000]fib(0) ->0
[0.00000000]fib(2) ->1
[0.00000000]fib(1) ->1
[0.00197220]fib(3) ->2
[0.00000000]fib(1) ->1
[0.00000000]fib(0) ->0
[0.00000000]fib(2) ->1
[0.00197220]fib(4) ->3
[0.00300097]fib(6) ->8


8

In [39]:
@functools.lru_cache()
@clock
def fib1(n):
    if n < 2:
        return n
    return fib1(n-1) + fib1(n-2)

fib1(6)

[0.00000000]fib1(1) ->1
[0.00000000]fib1(0) ->0
[0.00100088]fib1(2) ->1
[0.00100088]fib1(3) ->2
[0.00100088]fib1(4) ->3
[0.00100088]fib1(5) ->5
[0.00100088]fib1(6) ->8


8

### 7.8.2 单分派泛函数

In [40]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0}(0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

## 7.10 参数化装饰器
### 7.10.1 一个参数化的注册装饰器

In [41]:
registry = set()
def register(active = True):
    def decorate(func):
        print('running register(active = %s)->decorate(%s)' %(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active = False)
def f1():
    print('f1')
    
@register()
def f2():
    print('f2')
    
def f3():
    print('f3')

running register(active = False)->decorate(<function f1 at 0x000002093B7A03A8>)
running register(active = True)->decorate(<function f2 at 0x000002093B7A01F8>)


In [42]:
registry

{<function __main__.f2()>}

### 7.10.2 参数化colck装饰器

In [48]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s]{name}({args}) -> {result}'

def clock(fmt = DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def s(n):
    time.sleep(n)

s(3)

[3.00057340s]s(3) -> None


In [50]:
def f():
    _s = 1
    s = 2
    return locals()

In [51]:
f()

{'_s': 1, 's': 2}